In [1]:
from astrovision.data.satellite_image import (
    SatelliteImage,
)
from datetime import datetime, timezone

In [2]:
import s3fs
import os

fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])


In [3]:
import boto3
s3 = boto3.client("s3",endpoint_url = 'https://'+'minio.lab.sspcloud.fr',
                  aws_access_key_id= os.environ["AWS_ACCESS_KEY_ID"], 
                  aws_secret_access_key= os.environ["AWS_SECRET_ACCESS_KEY"], 
                  aws_session_token = os.environ["AWS_SESSION_TOKEN"])

In [18]:
# Recuperer les métas d'un fichier

bucket_name = "projet-slums-detection"
file = 'data-raw/PLEIADES/GUYANE_brut/2024_meta/IMG_PHR1A_PMS-N_202402251358470_ORT_PHR_PRO_FOP760ad51fbb517_20241008193335117_1_1_R1C3.tif'

existing_metadata = s3.head_object(Bucket=bucket_name, Key=file).get('Metadata', {})

# les bb
bb_image = existing_metadata['x-amz-meta-bounding_box']

In [4]:
# Définition du bucket et du dossier à lister
bucket_name = "projet-slums-detection"
s3_directory = "data-raw/PLEIADES/GUYANE_brut/2024/"

# Utilisation du paginator pour récupérer tous les fichiers
paginator = s3.get_paginator("list_objects_v2")
pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_directory)

# Récupération de tous les fichiers
file_keys = []
for page in pages:
    if "Contents" in page:
        file_keys.extend(obj["Key"] for obj in page["Contents"])

# Afficher le nombre total de fichiers trouvés
print(f"Nombre total de fichiers : {len(file_keys)}")



Nombre total de fichiers : 1375


In [54]:
bounds = {}
crs = {}
error_files = []
meta_files = []

for file_key in tqdm(file_keys):
    try :
        image = SatelliteImage.from_raster(
                            file_path=f"/vsis3/{bucket_name}/{file_key}",
                        )
    except:
        error_files.append(file_key)
    else:
        bounds[file_key] = str(image.bounds)
        crs[file_key] = str(image.crs)
        meta_files.append(file_key)


  3%|▎         | 38/1375 [06:21<2:20:16,  6.29s/it]

In [ ]:
s3_directory_meta = s3_directory[:-1] + "_meta/" 
for file_key in tqdm(meta_files):
    new_file = s3_directory_meta + file_key.split('/')[-1]

    # Nouvelles métadonnées à ajouter
    dt = s3.head_object(Bucket=bucket_name, Key=file_key).get('LastModified')
    formatted_date = str(dt.replace(microsecond=300000).isoformat())

    new_metadata = {
        "x-amz-meta-bounding_box": bounds[file_key],
        "x-amz-meta-date": formatted_date,
        "x-amz-meta-crs": crs[file_key]
    }

    # Étape 1 : Récupérer les métadonnées existantes
    existing_metadata = s3.head_object(Bucket=bucket_name, Key=file_key).get('Metadata', {})

    # Étape 2 : Ajouter les nouvelles métadonnées aux existantes
    combined_metadata = existing_metadata.copy()
    # Convertir les valeurs des tuples en chaînes
    combined_metadata["x-amz-meta-bounding_box"] = str(new_metadata["x-amz-meta-bounding_box"])
    combined_metadata["x-amz-meta-date"] = str(new_metadata["x-amz-meta-date"])
    combined_metadata["x-amz-meta-crs"] = str(new_metadata["x-amz-meta-crs"])

    
    # Étape 3 : Copier le fichier avec les nouvelles métadonnées
    s3.copy_object(
        Bucket=bucket_name,
        CopySource={'Bucket': bucket_name, 'Key': file_key},
        Key=new_file,  # Ou remplacer avec le même nom pour écraser l'objet
        Metadata=combined_metadata,
        MetadataDirective='REPLACE'  # Remplacer les métadonnées existantes
    )

